In [1]:
import gym
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

warnings.filterwarnings("ignore")

In [2]:
class QLearning:
    def __init__(self, bin_list, num_episodes, gamma, epsilon, max_steps, alpha):
        self.bin_list = bin_list
        self.num_episodes = num_episodes
        self.gamma = gamma
        self.epsilon = epsilon
        self.alpha = alpha  # 학습률 추가
        self.max_steps = max_steps

    def discretize(self, obs):
        idxs = [
            min(max(np.digitize(o, b) - 1, 0), len(b) - 1)
            for o, b in zip(obs, self.bin_list)
        ]
        return tuple(idxs)

    def decay_epsilon(self, rewards_log):
        if len(rewards_log) == 10:
            self.best_reward = np.mean(rewards_log[-10:])

        elif len(rewards_log) > 10 and len(rewards_log) % 100 == 0:
            avg_reward = np.mean(rewards_log[-10:])  # 최근 10개 보상 평균
            if avg_reward > self.best_reward:  # 최고 보상을 갱신하면 감소
                self.best_reward = avg_reward
                self.epsilon = max(0.01, self.epsilon - 0.001)

    def fit(self, env, Q):
        rewards_log = []

        for _ in tqdm(range(self.num_episodes)):
            obs, info = env.reset()
            state = self.discretize(obs)

            total_reward = 0

            for _ in range(self.max_steps):
                # ε-탐욕적으로 행동 선택
                if random.random() < self.epsilon:
                    action = random.randint(0, 1)
                else:
                    action = np.argmax(Q[state])

                obs_next, reward, done, truncated, info = env.step(action)
                next_state = self.discretize(obs_next)

                # Q-learning 업데이트 (max Q 사용)
                if done:
                    target = reward  # 종료 상태에서는 다음 상태의 Q값 없음
                else:
                    target = reward + self.gamma * np.max(Q[next_state])

                Q[state][action] += self.alpha * (target - Q[state][action])
                state = next_state  # SARSA와 다르게 다음 행동을 미리 선택하지 않음.

                if done or truncated:
                    break

            rewards_log.append(total_reward)
            self.decay_epsilon(rewards_log)

        env.close()
        return Q, rewards_log


In [3]:
# 1) 환경 & 이산화 준비
env = gym.make('CartPole-v1')
num_bins = 8
# 관측 범위
ranges = [(-4.8, 4.8), (-3, 3), (-0.418, 0.418), (-4, 4)]
bins_list = [np.linspace(lo, hi, num_bins) for (lo, hi) in ranges]

# 2) Q, 카운트(점진적 평균)
Q = defaultdict(lambda: np.zeros(env.action_space.n, dtype=np.float32))

In [4]:
QL = QLearning(bins_list, num_episodes=50000, gamma=0.99, epsilon=0.05, max_steps=500, alpha=0.1)
Q, rewards = QL.fit(env, Q)

 20%|█▉        | 5948/30000 [00:30<02:03, 194.65it/s] 

KeyboardInterrupt



In [ ]:
plt.figure(figsize=(15, 3), dpi=400)
plt.plot(np.array(rewards), label="Monte Carlo Method")
plt.title('CartPole-v1')
plt.xlabel('Episode', fontsize=15)
plt.ylabel('Reward', fontsize=15)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

print(f"최종 epsilon: {QL.epsilon}")
print("초반 10개 에피소드 보상:", rewards[:10])
print("마지막 10개 에피소드 보상:", rewards[-10:])
avg_reward_recent = np.mean(rewards[-100:])
print(f"최근 100 에피소드 평균 보상: {avg_reward_recent:.2f}")